In [18]:
import cv2 as cv
import numpy as np
import pandas as pd
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import nn
from torch import optim
import torch
from PIL import Image
import time
import random

In [19]:
class Dataset(Dataset):
    def __init__(self, csv_path, transform=None, augment_factor=1):
        self.data = pd.read_csv(csv_path)
        self.transform = transform

        unique_labels = sorted(self.data['id'].unique())
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

        self.data = pd.concat([self.data] * augment_factor, ignore_index=True)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['path']
        label_id = self.data.iloc[idx]['id']

        img = Image.open(img_path).convert("RGB")
        label = self.label_to_idx[label_id]

        if self.transform:
            img = self.transform(img)

        return img, label

In [3]:
class simpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(simpleCNN, self).__init__()

        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x)
        x = self.classifier(x)
        return x

In [20]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    print('using device:', device)
    for epoch in range(1, epochs+1):
        start_time = time.time()
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                output = model(inputs)
                loss = loss_fn(output,targets)
                valid_loss += loss.data.item() * inputs.size(0)
                correct = (output.argmax(dim=1) == targets)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'num_classes' : 528}, f'checkpoint/checkpoint#{epoch}.pth')
        end_time = time.time()
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}, time elapsed: {:.2f}'.format(epoch, training_loss, valid_loss, num_correct / num_examples, end_time - start_time)) 

In [21]:
def add_synthetic_noise(img_tensor):
    if random.random() < 0.5:
        noise = torch.randn_like(img_tensor) * 0.02
        img_tensor += noise
    return torch.clamp(img_tensor, 0, 1)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(0.8, 1.2)),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.2)], p=0.8),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Lambda(add_synthetic_noise),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

csv_path = 'mtgdb.csv'

train_data = Dataset(csv_path, train_transform, augment_factor=20)
val_data = Dataset(csv_path, val_transform, augment_factor=1)

train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [23]:
transfer_model = resnet18(weights=ResNet18_Weights.DEFAULT)
for name, param in transfer_model.named_parameters():
    if "bn" not in name:
        param.requires_grad = False
transfer_model.fc = nn.Sequential(
    nn.Linear(transfer_model.fc.in_features, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(512, 528)
)

In [ ]:
optimizer = optim.Adam(transfer_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transfer_model.to(device)

train(transfer_model, optimizer, loss_fn, train_data_loader, val_data_loader, epochs=10, device=device)

using device: cpu
Epoch: 1, Training Loss: 6.13, Validation Loss: 5.32, accuracy = 0.09, time elapsed: 689.41
Epoch: 2, Training Loss: 4.66, Validation Loss: 2.63, accuracy = 0.61, time elapsed: 675.85
Epoch: 3, Training Loss: 3.18, Validation Loss: 1.21, accuracy = 0.83, time elapsed: 750.10
Epoch: 4, Training Loss: 2.42, Validation Loss: 0.62, accuracy = 0.89, time elapsed: 758.11
Epoch: 5, Training Loss: 2.03, Validation Loss: 0.45, accuracy = 0.88, time elapsed: 766.25
Epoch: 6, Training Loss: 1.77, Validation Loss: 0.32, accuracy = 0.91, time elapsed: 754.16
Epoch: 7, Training Loss: 1.62, Validation Loss: 0.27, accuracy = 0.91, time elapsed: 751.03
Epoch: 8, Training Loss: 1.52, Validation Loss: 0.25, accuracy = 0.91, time elapsed: 751.45
Epoch: 9, Training Loss: 1.41, Validation Loss: 0.23, accuracy = 0.91, time elapsed: 748.74
Epoch: 10, Training Loss: 1.35, Validation Loss: 0.19, accuracy = 0.93, time elapsed: 750.04


: 